## Setup

In [1]:
using Distributed 
Distributed.addprocs()
Distributed.nprocs() # check if enough workers are available. should be n + 1, where n is # of logical cores. 

17

In [2]:
@everywhere using DifferentialEquations # will load on each core

In [4]:
# test on obsolete code for now
using StochasticDiffEq, DiffEqJump, Plots

struct AffectIndex{F1, F2}
    affect_index!::F1
    index::F2
  end

function (p::AffectIndex)(integrator)
      return p.affect_index!(integrator, p.index)
end
  
function f(du,u,p,t)
    du[1] = u[1]
end
  
function g(du,u,p,t)
    du[1] = u[1]
end
iv = [0.2]
prob = SDEProblem(f,g,iv,(0.0,10.0))

rate(u,p,t) = 2
affect_index!(integrator, index) = (integrator.u[index] = integrator.u[index]/2 )
#(integrator.u[index] = integrator.u[index]/2)
jumps = [ConstantRateJump(rate,AffectIndex(affect_index!, 1)) for i in 1:2 ]
jump_prob = JumpProblem(prob,Direct(),JumpSet((), jumps, nothing, nothing));

## Ensemble

In [23]:
propertynames(jump_prob.prob)

(:f, :g, :u0, :tspan, :p, :noise, :callback, :noise_rate_prototype, :seed)

In [24]:
# alter the problem initial condition with each run 
@everywhere function prob_func(prob,i,repeat) # different initial conditions
  new_prob = SDEProblem(prob.prob.f, prob.prob.g, rand()*prob.prob.u0, prob.prob.tspan)
  JumpProblem(new_prob,Direct(),JumpSet((), jumps, nothing, nothing))
end

In [100]:
@everywhere using StatsBas

ArgumentError: ArgumentError: Package Iterators not found in current path:
- Run `import Pkg; Pkg.add("Iterators")` to install the Iterators package.


In [97]:
# try it with both
@everywhere function reduction_func(u, data, i)
    sols = [datum.u for datum in data];
    u = append!(u, summarystats(flatten(sols)))
    return (u, false)
end 

In [98]:
ensemble_prob = EnsembleProblem(jump_prob;
                prob_func= prob_func,
                reduction = reduction_func,
                u_init = [])

EnsembleProblem with problem JumpProblem

In [99]:
sim = solve(ensemble_prob, trajectories = 1000, batch_size = 1000);

UndefVarError: UndefVarError: flatten not defined

In [ ]:
sim.